In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import sys
sys.path.append('/content/drive/Shareddrives/Kickstar_team3')
from sjsu_OD import ObjectDetector
import io
import os
import json
import torch
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
import cv2
from transformers import AutoModelForCausalLM, AutoProcessor
from peft import LoraConfig, get_peft_model
import random


In [ ]:
import shutil

zip_file_path = "/content/drive/Shareddrives/Kickstar_team3/epoch_20-20250402T170926Z-001.zip"

extract_to_path = "finetuned1"

shutil.unpack_archive(zip_file_path, extract_to_path)
print(f"Unzipped files are available at: {extract_to_path}")

Unzipped files are available at: finetuned1


In [ ]:
import json

def convert_to_florence2_format(input_json, output_json):
    with open(input_json, "r") as file:
        data = json.load(file)

    with open(output_json, "w") as file:
        file.write('{"annotations": [\n')  # Start JSON structure

        first_entry = True
        for image_path, obj_data in data.items():
            label = obj_data["label"]
            valid_bboxes = []

            for bbox in obj_data["bboxes"]:
                x_min, y_min, x_max, y_max = map(int, bbox)  # Convert all elements to int
                if x_min >= 2 and x_max < 638:
                    width, height = int(x_max - x_min), int(y_max - y_min)
                    valid_bboxes.append({"bbox": [x_min, y_min, width, height], "bbox_label": label})

            if len(valid_bboxes) == 2:
                continue

            if valid_bboxes:
                if not first_entry:
                    file.write(",\n")  # Add comma between entries
                json.dump({"image": image_path, "instances": valid_bboxes}, file)
                first_entry = False

        file.write("\n]}")  # Close JSON structure



In [ ]:
model_name ='/content/finetuned1/epoch_20'
object_detector = ObjectDetector(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


processing_florence2.py:   0%|          | 0.00/46.4k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Florence-2-base:
- processing_florence2.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


config.json:   0%|          | 0.00/2.43k [00:00<?, ?B/s]

configuration_florence2.py:   0%|          | 0.00/15.1k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Florence-2-base:
- configuration_florence2.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_florence2.py:   0%|          | 0.00/127k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Florence-2-base:
- modeling_florence2.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
/usr/local/lib/python3.11/dist-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


pytorch_model.bin:   0%|          | 0.00/464M [00:00<?, ?B/s]

In [ ]:


video_path = "/content/drive/Shareddrives/Kickstar_team3/Connor_Building_Video/Ceasar_Chavez_Memorial.MOV"
output_video_path = "output_video.mp4"
output_frame_dir = "/content/drive/Shareddrives/Kickstar_team3/Connor_Building_Video/frame_memorial"
skip_frames=6
answer="Ceasar_Chavez_Memorial"
output_data_path = f"/content/drive/Shareddrives/Kickstar_team3/Connor_Building_Video/{answer}.json"

all_frame_data = object_detector.process_video(video_path, output_frame_dir, answer, skip_frames, None)
object_detector.save_detection_data(all_frame_data, output_data_path, answer)


In [ ]:
input_json = "/content/drive/Shareddrives/Kickstar_team3/Connor_Building_Video/Ceasar_Chavez_Memorial.json"
output_json = "/content/drive/Shareddrives/Kickstar_team3/Connor_Building_Video/Ceasar_Chavez_Memorial1.json"

convert_to_florence2_format(input_json, output_json)